In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import glob
!pip install openpyxl

day_folder = "/content/drive/My Drive/8 juni/"
file_paths = glob.glob(day_folder + "*.csv")

school_id = 45
class_id = 105

dataframes = [pd.read_csv(file, sep=';') for file in file_paths]
keyfile_general = pd.read_csv("Keyfile_csv.csv")
keyfile_specific = pd.read_excel("keyfile school 45 class 105.xlsx")


file_to_subject = {}
teacher_tag = '0x24046130BA41'


for file in file_paths:
    df_temp = pd.read_csv(file, sep=';')
    tag_id = df_temp['TagId'].iloc[0]  # This is the TrackLabID
    df = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    try:

        subject_id = keyfile_specific.loc[keyfile_specific['trackingnnumer'] == tagnr, 'subject ID'].values[0] #added a space
    except:
        print("Tag not found:" + tag_id)
        print(tagnr)
        subject_id = None

    # Store TrackLabID (tag_id), Tagnumber, and subject ID in the dictionary
    file_to_subject[tag_id] = {'Tagnumber': tagnr, 'subject ID': subject_id}
#print(file_to_subject)

# Filter dataframes to remove unfound tags (except teacher)
filtered_dataframes = [
    df for df in dataframes
    if (file_to_subject.get(df['TagId'].iloc[0]) and
        file_to_subject[df['TagId'].iloc[0]]['subject ID'] is not None) or
       df['TagId'].iloc[0] == teacher_tag
]

dataframes = filtered_dataframes  # Replace original dataframes with filtered ones


Mounted at /content/drive
Tag not found:0x24025F449EFB
30


In [2]:


def process_timestamps(df):
    # Extract 'TrackLabID'
    tag_id = df['TagId'].iloc[0]

    try:
        df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], format='%m/%d/%Y %H:%M:%S.%f').dt.floor('s')
    except ValueError:
        # Handle cases where 'TimeStamp' is already in datetime format
        df['TimeStamp'] = pd.to_datetime(df['TimeStamp']).dt.floor('s')

    # Keep only the first entry for each second
    df = df.groupby('TimeStamp').first().reset_index()

    for column in ['X', 'Y', 'Z']:
        # Iteratively detects and replaces outliers until no more are found
        while True:
            # Calculates the difference between consecutive values
            diff = df[column].diff()

            # Identify outliers where the difference is greater than 1.5
            outlier_indices = diff[abs(diff) > 1.5].index

            # loop break condition
            if len(outlier_indices) == 0:
                break

            # Replace outliers with the prior value
            for index in outlier_indices:
                if index > 0:  # >0 avoids first row (which has no prior value)
                    df.loc[index, column] = df.loc[index - 1, column]

    # Convert 'TimeStamp' to datetime in all dataframes
    for df_ in dataframes:
        df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%m/%d/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime

    time_range = pd.date_range(
        start=min(df_['TimeStamp'].min() for df_ in dataframes),
        end=max(df_['TimeStamp'].max() for df_ in dataframes),
        freq='s'
    )
    full_df = pd.DataFrame({'TimeStamp': time_range})

    # fills missing seconds
    df = pd.merge(full_df, df, left_on='TimeStamp', right_on='TimeStamp', how='left')

    # Forward fill and then backward fill to handle NaNs at the beginning and end
    df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']].ffill().bfill()
    # Interpolate any remaining NaNs
    df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']].interpolate(method='linear')

    # Add 'TagID' column back to the DataFrame
    df['TagId'] = tag_id
    return df

dataframes = [df for df in [process_timestamps(df) for df in dataframes] if df is not None]



<ipython-input-2-5d378172a795>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%m/%d/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime
<ipython-input-2-5d378172a795>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df_['TimeStamp'], format='%m/%d/%Y %H:%M:%S.%f', errors='ignore').dt.floor('s')  #errors='ignore' to avoid errors if the column is already datetime
<ipython-input-2-5d378172a795>:34: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df_['TimeStamp'] = pd.to_datetime(df_['

In [3]:
import pandas as pd

def find_extremes_and_context(df, column):
    """
    Manual check as outlier detection, was used to determine 1.5 as sensible value for allowed value jumps.
    """

    # Find index of minimum and maximum values
    min_index = df[column].idxmin()
    max_index = df[column].idxmax()

    # Print +-5 values around min/max
    print(f"Minimum value of {column}:")
    print(df.loc[max(0, min_index - 5):min(len(df), min_index + 5), ['TimeStamp', 'X', 'Y', 'Z']])

    print(f"\nMaximum value of {column}:")
    print(df.loc[max(0, max_index - 5):min(len(df), max_index + 5), ['TimeStamp', 'X', 'Y', 'Z']])


for df in dataframes:
    for column in ['X', 'Y']:
        find_extremes_and_context(df, column)

Minimum value of X:
               TimeStamp         X         Y         Z
1220 2023-06-08 10:26:10  0.332111  3.257516  0.402544
1221 2023-06-08 10:26:11  0.332111  3.257516  0.402544
1222 2023-06-08 10:26:12  0.332111  3.257516  0.402544
1223 2023-06-08 10:26:13  0.332111  3.257516  0.402544
1224 2023-06-08 10:26:14  0.332111  3.257516  0.242586
1225 2023-06-08 10:26:15 -0.559444  3.334812  0.347396
1226 2023-06-08 10:26:16 -0.226841  3.458713  0.375752
1227 2023-06-08 10:26:17 -0.226841  3.458713  0.376034
1228 2023-06-08 10:26:18 -0.226841  3.458713  0.376034
1229 2023-06-08 10:26:19 -0.226841  3.458713  0.376034
1230 2023-06-08 10:26:20 -0.226841  3.458713  0.376034

Maximum value of X:
               TimeStamp         X         Y         Z
3509 2023-06-08 11:04:19  5.415098  5.344249  0.584971
3510 2023-06-08 11:04:20  5.415098  5.344249  0.584971
3511 2023-06-08 11:04:21  5.415098  5.344249  0.584971
3512 2023-06-08 11:04:22  5.415098  5.344249  0.584971
3513 2023-06-08 11:04:23

In [4]:
#don't actually use the averages, could be used for plotting though (I removed those cells from the notebook because we don't actually need it, could add back if you want)
object_averages = {}

for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')


    if tag_id == teacher_tag:
        tagnr = '35'
        subject_id = "Teacher"

    elif tag_id in df['TrackLabID'].values:
        tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
        subject_id = keyfile_specific.loc[keyfile_specific['trackingnnumer'] == tagnr, 'subject ID'].values[0]




    avg_x, avg_y, avg_z = df_[['X', 'Y', 'Z']].mean()
    object_averages[f"Person_{subject_id} with Tagnr {tagnr}"] = (avg_x, avg_y, avg_z)

avg_df = pd.DataFrame.from_dict(object_averages, orient="index", columns=["X", "Y", "Z"])
print(avg_df)

                                     X         Y         Z
Person_18.0 with Tagnr 7      2.869693  3.295815  0.424384
Person_15.0 with Tagnr 25     5.614148  1.753287  0.486479
Person_Teacher with Tagnr 35  4.553121  3.901698  0.336515
Person_7.0 with Tagnr 18      4.148503  1.930445  0.321734
Person_3.0 with Tagnr 11      1.567302  3.471072  0.391006
Person_8.0 with Tagnr 21      2.981511  4.139417  0.416807
Person_21.0 with Tagnr 19     3.987221  1.604122  0.421370
Person_16.0 with Tagnr 24     4.662008  1.571138  0.517143
Person_1.0 with Tagnr 22      5.475533  3.488271  0.278904
Person_13.0 with Tagnr 26     5.158634  3.883294  0.532516
Person_22.0 with Tagnr 15     3.169241  1.600694  0.406505
Person_11.0 with Tagnr 3      1.201732  4.616444  0.422101
Person_14.0 with Tagnr 9      3.278108  2.782913  0.403194
Person_20.0 with Tagnr 27     3.954748  3.219837  0.305173
Person_9.0 with Tagnr 13      4.257100  1.296487  0.486224
Person_5.0 with Tagnr 2       0.884644  4.231346  0.4200

In [5]:
fivemin_object_averages = {}
for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')


    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
    # Calculate average for the first 300 rows (= 5 minutes)
    avg_x, avg_y, avg_z = df_[['X', 'Y', 'Z']].head(300).mean()
    fivemin_object_averages[f"{tagnr}"] = (avg_x, avg_y, avg_z)

avg_df = pd.DataFrame.from_dict(fivemin_object_averages, orient="index", columns=["X", "Y", "Z"])

print(avg_df)

           X         Y         Z
7  -0.039775  3.278581  0.406610
25  5.608412  1.896083  0.383260
35  4.174585  3.946842  0.259681
18  4.224041  2.725275  0.363865
11  1.378351  3.281498  0.547523
21  2.922858  3.873522  0.465759
19  3.999931  3.586043  0.434404
24  5.283935  1.946278  0.499090
22  5.323469  3.485513  0.309308
26  5.616456  3.713150  0.583257
15  3.253835  1.716569  0.461957
3   1.144444  4.123365  0.433304
9   0.952192  2.239659  0.446726
27  2.782352  3.686216  0.217827
13  4.337819  1.687585  0.450243
2   0.867616  4.419237  0.465361
6   4.689494  4.342684  0.190193
20  4.496042  4.099894  0.539823
5   1.565765  4.475087  0.368322
17  2.830320  2.230072  0.367449
10  0.425597  2.262980  0.524494


In [6]:
#this initially was for baseline distance purposes, decided using avg first 5 minutes might be better though due to trackers potentially not being turned on in usual positions (e.g. turning it on whilst picking it up from teachers desk)
object_start = {}

for index, df_ in enumerate(dataframes):
    tag_id = df_['TagId'].iloc[0]
    df = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df.loc[df['TrackLabID'] == tag_id, 'Tagnumber'].values[0]
    # respective 1st coordinates
    start_x = df_['X'].iloc[0]
    start_y = df_['Y'].iloc[0]
    start_z = df_['Z'].iloc[0]
    object_start[f"{tagnr}"] = (start_x, start_y, start_z)

start_positions_df = pd.DataFrame.from_dict(object_start, orient="index", columns=["X", "Y", "Z"])

print(start_positions_df)

           X         Y         Z
7  -0.274216  3.330947  0.538307
25  5.469565  1.508611  0.371988
35  4.174585  3.946842  0.259681
18  4.040825  2.746120  0.222364
11  1.239972  3.262114  0.496307
21  2.897379  3.932407  0.380791
19  4.025272  3.682546  0.334470
24  5.280248  2.076571  0.465271
22  5.323469  3.485513  0.309308
26  5.616456  3.713150  0.583257
15  3.176772  1.688070  0.261040
3   1.104457  4.131473  0.258799
9   0.952192  2.239659  0.575319
27  2.782352  3.686216  0.217827
13  4.216166  1.400529  0.445318
2   0.631426  4.366888  0.595047
6   4.689494  4.342684  0.190193
20  4.443018  4.363239  0.356490
5   1.561693  4.474602  0.375869
17  3.007332  2.253563  0.232472
10  0.257081  2.331370  0.842258


In [7]:
import numpy as np

def euclidean_distance(x1, y1, x2, y2):
  return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

target_tagnr = 27

# Find target coordinates using tagnr
target_coords = None
for tracker, (x, y, _) in fivemin_object_averages.items():
  if tracker == f"{target_tagnr}":
    target_coords = (x, y)
    break




target_x, target_y = target_coords
min_distance = float('inf')
closest_tracker = None

for tracker, (x, y, _) in fivemin_object_averages.items():
  if tracker != f"{target_tagnr}":  # Excludes teacher who would have distance 0
    distance = euclidean_distance(target_x, target_y, x, y)
    print(distance, tracker) #can be removed later
    if distance < min_distance:
      min_distance = distance
      closest_tracker = tracker

closest_distance_to_35 = min_distance
print(f"The closest tracker to the teacher is {closest_tracker} with a distance of {closest_distance_to_35}")

2.8514149378440576 7
3.3453234702463543 25
1.4164182516869843 35
1.7325918326056948 18
1.4611685734465645 11
0.23414912045530234 21
1.2216928090048191 19
3.0471787233583587 24
2.5490304368553316 22
2.8342324165637236 26
2.025291397692645 15
1.6952412014856384 3
2.33281233743248 9
2.5325881257054976 13
2.050252054172058 2
2.0169637554219513 6
1.7629130290626056 20
1.449966418900066 5
1.4569335358210638 17
2.7531608845695015 10
The closest tracker to the teacher is 21 with a distance of 0.23414912045530234


In [8]:
tracker_counts = {}
total_teacher_proximity_count = 0
teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher
print(len(teacher_df))
for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Initialise count for tracker
    tracker_counts[f"{tagnr}"] = 0


    # Iterate through rows using the index
    for index in range(len(df)):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < closest_distance_to_35:
            tracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
sorted_tracker_counts = dict(sorted(tracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within full distance of anyone: {total_teacher_proximity_count}")


exp_amount = 100 / (len(dataframes)-1) #determines expected proximity percentage if it were fully equal(e.g. 20 students = 5%), -1 due to not counting teacher
lower = 0.75
upper = 1.25
for tracker, count in sorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within full distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

8816
Total times teacher was within full distance of anyone: 12877
Person nan with Tracker 35: 8816 times within full distance of teacher, category: above average with proximity percentage 68.46
Person 18.0 with Tracker 7: 3371 times within full distance of teacher, category: above average with proximity percentage 26.18
Person 14.0 with Tracker 9: 612 times within full distance of teacher, category: average with proximity percentage 4.75
Person 21.0 with Tracker 19: 53 times within full distance of teacher, category: below average with proximity percentage 0.41
Person 13.0 with Tracker 26: 14 times within full distance of teacher, category: below average with proximity percentage 0.11
Person 7.0 with Tracker 18: 7 times within full distance of teacher, category: below average with proximity percentage 0.05
Person 8.0 with Tracker 21: 2 times within full distance of teacher, category: below average with proximity percentage 0.02
Person 1.0 with Tracker 22: 1 times within full distance 

In [9]:
half_distance = closest_distance_to_35 / 2
htracker_counts = {}
total_teacher_proximity_count = 0

teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher


for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Skip unmatchable trackers
    if file_to_subject.get(tag_id) and file_to_subject[tag_id]['subject ID'] is None:
        continue

    # Initialise count for tracker
    htracker_counts[f"{tagnr}"] = 0

    # Get minimum length of both dataframes to avoid IndexError
    min_len = min(len(df), len(teacher_df))

    # Iterate through rows using the index
    for index in range(min_len):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < half_distance:
            htracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
hsorted_tracker_counts = dict(sorted(htracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within half distance of anyone: {total_teacher_proximity_count}")
# Print the sorted counts with subject IDs


for tracker, count in hsorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within half distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

Total times teacher was within half distance of anyone: 12213
Person nan with Tracker 35: 8816 times within half distance of teacher, category: above average with proximity percentage 72.19
Person 18.0 with Tracker 7: 3371 times within half distance of teacher, category: above average with proximity percentage 27.60
Person 14.0 with Tracker 9: 12 times within half distance of teacher, category: below average with proximity percentage 0.10
Person 13.0 with Tracker 26: 9 times within half distance of teacher, category: below average with proximity percentage 0.07
Person 7.0 with Tracker 18: 4 times within half distance of teacher, category: below average with proximity percentage 0.03
Person 1.0 with Tracker 22: 1 times within half distance of teacher, category: below average with proximity percentage 0.01
Person 15.0 with Tracker 25: 0 times within half distance of teacher, category: below average with proximity percentage 0.00
Person 3.0 with Tracker 11: 0 times within half distance of

In [10]:
quarter_distance = closest_distance_to_35/4

qtracker_counts = {}
total_teacher_proximity_count = 0

teacher_df = next((df for df in dataframes if df['TagId'].iloc[0] == teacher_tag), None)  # Find teacher


for df in dataframes:
    tag_id = df['TagId'].iloc[0]
    df_keyfile = pd.read_csv("Keyfile_csv.csv", sep=';')
    tagnr = df_keyfile.loc[df_keyfile['TrackLabID'] == tag_id, 'Tagnumber'].values[0]

    # Skip teacher
    if tagnr == target_tagnr:
        continue

    # Skip unmatchable trackers
    if file_to_subject.get(tag_id) and file_to_subject[tag_id]['subject ID'] is None:
        continue

    # Initialise count for tracker
    qtracker_counts[f"{tagnr}"] = 0

    # Get minimum length of both dataframes to avoid IndexError
    min_len = min(len(df), len(teacher_df))

    # Iterate through rows using the index
    for index in range(min_len):
        x = df.loc[index, 'X']
        y = df.loc[index, 'Y']

        # Get teacher's coordinates from the same row index
        target_x = teacher_df.loc[index, 'X']
        target_y = teacher_df.loc[index, 'Y']

        # Calculate distance to target
        distance = euclidean_distance(target_x, target_y, x, y)

        # Check if within desired distance
        if distance < quarter_distance:
            qtracker_counts[f"{tagnr}"] += 1
            total_teacher_proximity_count += 1

    # Sort tracker counts
qsorted_tracker_counts = dict(sorted(qtracker_counts.items(), key=lambda item: item[1], reverse=True))

print(f"Total times teacher was within quarter distance of anyone: {total_teacher_proximity_count}")
# Print the sorted counts with subject IDs


for tracker, count in qsorted_tracker_counts.items():
    subject_id = file_to_subject.get(df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0], {}).get('subject ID')
    proximity_percentage = (count / total_teacher_proximity_count) * 100 #how much % of total teacher-student proximity was with this student
    ratio = proximity_percentage/exp_amount #ratio above 1 means more encounters than avg, below 1 less

    if ratio < lower:
        category = "below average"
    elif ratio > upper:
        category = "above average"
    else:
        category = "average"
    print(f"Person {subject_id} with Tracker {tracker}: {count} times within quarter distance of teacher, category: {category} with proximity percentage {proximity_percentage:.2f}")

Total times teacher was within quarter distance of anyone: 8828
Person nan with Tracker 35: 8816 times within quarter distance of teacher, category: above average with proximity percentage 99.86
Person 14.0 with Tracker 9: 5 times within quarter distance of teacher, category: below average with proximity percentage 0.06
Person 13.0 with Tracker 26: 4 times within quarter distance of teacher, category: below average with proximity percentage 0.05
Person 7.0 with Tracker 18: 2 times within quarter distance of teacher, category: below average with proximity percentage 0.02
Person 1.0 with Tracker 22: 1 times within quarter distance of teacher, category: below average with proximity percentage 0.01
Person 18.0 with Tracker 7: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 15.0 with Tracker 25: 0 times within quarter distance of teacher, category: below average with proximity percentage 0.00
Person 3.0 with Tracker 11: 0 times withi

In [12]:

merge_df = pd.read_excel("merged-data-WIP_2025-05-01_17-31 (1) (1) (1) (1).xlsx")  # Replace with your file name

# Create new columns if they don't exist
for distance_type in ['full', 'half', 'quarter']:
    if f'{distance_type}_proximity' not in merge_df.columns:
        merge_df[f'{distance_type}_proximity'] = None
    if f'{distance_type}_category' not in merge_df.columns:
        merge_df[f'{distance_type}_category'] = None


for distance_type, sorted_counts in [('full', sorted_tracker_counts), ('half', hsorted_tracker_counts), ('quarter', qsorted_tracker_counts)]:
    total_count = sum(sorted_counts.values()) # total proximity count for this distance type

    for tracker, count in sorted_counts.items():

        tag_id = df_keyfile.loc[df_keyfile['Tagnumber'] == int(tracker), 'TrackLabID'].values[0]

        # Find the matching row in the existing DataFrame
        mask = (merge_df['school'] == school_id) & (merge_df['class'] == class_id) & (merge_df['tracklab_id'] == tag_id)  # Replace 'tracklab_id' with the actual column name for TagID if different
        matching_row_index = merge_df[mask].index

        # Update the existing DataFrame if matching row is found
        if len(matching_row_index) > 0:
            row_index = matching_row_index[0]
            merge_df.loc[row_index, f'{distance_type}_proximity'] = (count / total_count) * 100

            proximity_percentage = (count / total_count) * 100
            ratio = proximity_percentage / exp_amount

            if ratio < lower:
                category = "below average"
            elif ratio > upper:
                category = "above average"
            else:
                category = "average"

            merge_df.loc[row_index, f'{distance_type}_category'] = category


merge_df.to_excel("merged-data-WIP_2025-05-01_17-31 (1) (1) (1) (1).xlsx", index=False)